# Step 1 :Import Dataset
In the code cell below, we import a dataset of actress images. We populate a few variables through the use of the load_files function from the scikit-learn library:

1. train_files, valid_files, test_files - numpy arrays containing file paths to images
2. train_targets, valid_targets, test_targets - numpy arrays containing onehot-encoded classification labels
3. actress_names - list of string-valued dog breed names for translating labels

In [17]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from pprint import pprint
#define function to load train, test and valid datasets
def load_dataset(path):
    data = load_files(path)
    actress_files = np.array(data['filenames'])
    actress_targets = np_utils.to_categorical(np.array(data['target']), 5) #As of now we have 5 actress to comapre with
    return actress_files, actress_targets

#load train,test,valid datasets
train_files, train_targets = load_dataset('../Celebs/train')
test_files, test_targets = load_dataset('../Celebs/test')
valid_files, valid_targets = load_dataset('../Celebs/valid')

print("Total Actress Images {}".format(len(np.hstack([train_files, test_files, valid_files]))))
print("Train Actress Images {}".format(len(train_files)))
print("{} ->{}".format(train_files[0], train_targets[0]))
print("{} ->{}".format(train_files[1000], train_targets[1000]))
print("{} ->{}".format(train_files[2000], train_targets[2000]))
print("{} ->{}".format(train_files[3000], train_targets[3000]))
print("Test Actress Images {}".format(len(test_files)))
print("{} ->{}".format(test_files[0], test_targets[0]))
print("{} ->{}".format(test_files[100], test_targets[100]))
print("{} ->{}".format(test_files[185], test_targets[185]))
print("{} ->{}".format(test_files[250], test_targets[250]))
print("Valid Actress Images {}".format(len(valid_files)))

Total Actress Images 4970
Train Actress Images 3978
../Celebs/train/anushka/1546.jpg ->[1. 0. 0. 0. 0.]
../Celebs/train/kat/1481.jpg ->[0. 0. 0. 1. 0.]
../Celebs/train/priyanka/147.jpg ->[0. 0. 0. 0. 1.]
../Celebs/train/jaq/1383.jpg ->[0. 0. 1. 0. 0.]
Test Actress Images 496
../Celebs/test/anushka/376.jpg ->[1. 0. 0. 0. 0.]
../Celebs/test/anushka/52.jpg ->[1. 0. 0. 0. 0.]
../Celebs/test/kat/1173.jpg ->[0. 0. 0. 1. 0.]
../Celebs/test/priyanka/732.jpg ->[0. 0. 0. 0. 1.]
Valid Actress Images 496


# Pre-process the data
Here we are using tensorflow as backend for keras and it requires our images as a certain 4D array a.k.a 4D Tensor with shape.

$$
(\text{nb_samples}, \text{rows}, \text{columns}, \text{channels}),
$$

where nb_samples corresponds to the total number of images (or samples), and rows, columns, and channels correspond to the number of rows, columns, and channels for each image, respectively.

The path_to_tensor takes a string spacifying file location and it does the following operation.

1. Resizes the image as (224,224).
2. Convert the squared image as an array (3d array)
3. Expand the 3d array to 4d array as (1,224,224,3)

Another helper function paths_to_tensor takes an array of image file locations as param and in turn calls path_to_tensor on all of them and then vertically stack the output.

Here, nb_samples is the number of samples, or number of images, in the supplied array of image paths. It is best to think of nb_samples as the number of 3D tensors (where each 3D tensor corresponds to a different image) in your dataset!

In [18]:
from keras.preprocessing import image
from tqdm import tqdm_notebook as tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)
    

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]    
    return np.vstack(list_of_tensors)

In [19]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True   

#pre-process the data
train_tensors = paths_to_tensor(train_files).astype('float32')/255
print(train_tensors.shape)
test_tensors = paths_to_tensor(test_files).astype('float32')/255
print(test_tensors.shape)
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
print(valid_tensors.shape)


(3978, 224, 224, 3)



(496, 224, 224, 3)



(496, 224, 224, 3)


# TODO
1. Create a CNN from scratch with these tensors
2. Create Augmentations and with use of that, And create bottleneck features and save it to S3
3. Fetch bottleneck features from S3 and use transfer learning to build a CNN

# Step 2 Create CNN from the scratch
Here we will create a CNN from the scratch using Keras. Note that we will not be using any models (vgg,resnet) we will take images as input and train our model.

Our target here is to create simplistic model with training,test accuracy above 5%.

Be careful with adding too many trainable layers! More parameters means longer training, which means you are more likely to need a GPU to accelerate the training process. Thankfully, Keras provides a handy estimate of the time that each epoch is likely to take; you can extrapolate this estimate to figure out how long it will take for your algorithm to train.


In [20]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

### TODO: Define your architecture.
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(GlobalAveragePooling2D())
# model.add(Flatten())
model.add(Dense(train_targets.shape[1], activation='softmax'))
model.add(Dropout(0.4))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
global_average_pooling2d_2 ( (None, 64)                0         
__________

### Compile the Model

In [22]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [23]:
from keras.callbacks import ModelCheckpoint

epochs = 50

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, validation_data=(valid_tensors, valid_targets),
         epochs=50, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 3978 samples, validate on 496 samples
Epoch 1/50
3978/3978 [==============================] - 12s 3ms/step - loss: 0.1616 - acc: 0.3064 - val_loss: 1.1921e-07 - val_acc: 0.3085

Epoch 00001: val_loss improved from inf to 0.00000, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 2/50
3978/3978 [==============================] - 11s 3ms/step - loss: 1.1921e-07 - acc: 0.3082 - val_loss: 1.1921e-07 - val_acc: 0.3085

Epoch 00002: val_loss did not improve from 0.00000
Epoch 3/50
3978/3978 [==============================] - 11s 3ms/step - loss: 1.1921e-07 - acc: 0.3082 - val_loss: 1.1921e-07 - val_acc: 0.3085

Epoch 00003: val_loss did not improve from 0.00000
Epoch 4/50
3978/3978 [==============================] - 11s 3ms/step - loss: 1.1921e-07 - acc: 0.3082 - val_loss: 1.1921e-07 - val_acc: 0.3085

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/50
1160/3978 [=======>......................] - ETA: 7s - loss: 1.1921e-07 - acc: 0.3233

KeyboardInterrupt: 